[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/comfyui-colab/blob/main/api/raemuXL_v35Lightning.ipynb)

In [ ]:
%cd /content
!git clone https://github.com/camenduru/KaonashiUI
%cd /content/KaonashiUI
!pip install xformers==0.0.26.post1 torchsde==0.2.6 einops==0.8.0 diffusers==0.28.0 transformers==4.41.2 accelerate==0.30.1
!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/TostModels/resolve/main/models/checkpoints/raemuXL_v35Lightning.safetensors -d /content/KaonashiUI/models/checkpoints -o model.safetensors
import torch
import random
from kaonashi.sd import load_checkpoint_guess_config
import nodes
import numpy as np
from PIL import Image
with torch.inference_mode():
    model_patcher, clip, vae, clipvision = load_checkpoint_guess_config("/content/KaonashiUI/models/checkpoints/model.safetensors", output_vae=True, output_clip=True, embedding_directory=None)

In [ ]:
with torch.inference_mode():
    height = 1024 # @param {type:"number"}
    width = 1024 # @param {type:"number"}
    seed = 0 # @param {type:"number"}
    steps = 9 # @param {type:"number"}
    cfg = 2.0 # @param {type:"number"}
    sampler_name = "euler" # @param {type:"string"}
    scheduler = "normal" # @param {type:"string"}
    latent = {"samples":torch.zeros([1, 4, height // 8, width // 8])}
    prompt= "(masterpiece, best quality, very aesthetic, ultra detailed), intricate details, 1girl, esdeath, akame ga kill!, blue eyes, blue hair, military uniform, hat, ice, sitting, sitting on throne, crossed legs, snow, white footwear, thigh boots," # @param {type:"string"}
    cond, pooled = clip.encode_from_tokens(clip.tokenize(prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]
    negative_prompt = "(worst quality, low quality, very displeasing, lowres), (interlocked fingers, badly drawn hands and fingers, anatomically incorrect hands), blurry, watermark," # @param {type:"string"}
    n_cond, n_pooled = clip.encode_from_tokens(clip.tokenize(negative_prompt), return_pooled=True)
    n_cond = [[n_cond, {"pooled_output": n_pooled}]]
    if seed == 0:
        seed = random.randint(0, 18446744073709551615)
    print(seed)
    sample = nodes.common_ksampler(model=model_patcher, seed=seed, steps=9, cfg=2.0, sampler_name="euler", scheduler="normal", positive=cond, negative=n_cond, latent=latent, denoise=1)
    sample = sample[0]["samples"].to(torch.float16)
    vae.first_stage_model.cuda()
    decoded = vae.decode_tiled(sample).detach()
Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])